In [1]:
# import
from arm import BernoulliArm
from greedy import Greedy

<BernoulliArm> mu: 0.8 hit/trial: 0/0
<Greedy> N=1000 Arms=[BernoulliArm(0.1), BernoulliArm(0.6), BernoulliArm(0.8), BernoulliArm(0.2)]


In [5]:
def mean_cumulative_reward(df_list):
    """累積報酬の平均を算出する
    
    :param df_list: DataFrameのリスト
    :return reward
    """
    return sum([df["reward"].sum() for df in df_list])

In [24]:
def hit_rate(df_list):
    """各時刻の当たり率を時系列リストにして返す
    
    :param df_list: DataFrameのリスト
    :return hit_rate_list: list of hit rate (float, 0~1)
    """
    return list(sum([df["reward"] for df in df_list])/len(df_list))

SyntaxError: unexpected EOF while parsing (<ipython-input-24-1b1985803158>, line 7)

In [25]:
def best_arm_rate(df_list,best_arm):
    """各時刻の最適アーム選択率を時系列リストにして返す
    :param df_list: DataFrameのリスト
    :param best_arm: (int) 最適なアームのインデックス(最適なアームとは報酬の期待値が最大のアーム)
    :return best_arm_rate_list: list of best arm rate (float, 0~1)
    """
    for df in df_list:
        df["is_best"]=df["arm"].apply(lambda x: 1 if x==best_arm  else 0)
    print(df[])
    print()
    return 

In [ ]:
mus = [0.1, 0.6, 0.8, 0.2]
arms = [BernoulliArm(mu) for mu in mus]
N=1000
alg = Greedy(arms,N)
print(alg)

In [2]:
df = alg.execute()

In [6]:
mean_cumulative_reward([df])

637

In [15]:
hit_rate([df,df])


1000

In [26]:
best_arm_rate([df,df],2)

[0      0
 1      0
 2      1
 3      0
 4      0
 5      0
 6      0
 7      0
 8      0
 9      0
 10     0
 11     0
 12     0
 13     0
 14     0
 15     0
 16     0
 17     0
 18     0
 19     0
 20     0
 21     0
 22     0
 23     0
 24     0
 25     0
 26     0
 27     0
 28     0
 29     0
       ..
 970    0
 971    0
 972    0
 973    0
 974    0
 975    0
 976    0
 977    0
 978    0
 979    0
 980    0
 981    0
 982    0
 983    0
 984    0
 985    0
 986    0
 987    0
 988    0
 989    0
 990    0
 991    0
 992    0
 993    0
 994    0
 995    0
 996    0
 997    0
 998    0
 999    0
 Name: is_best, Length: 1000, dtype: int64, 0      0
 1      0
 2      1
 3      0
 4      0
 5      0
 6      0
 7      0
 8      0
 9      0
 10     0
 11     0
 12     0
 13     0
 14     0
 15     0
 16     0
 17     0
 18     0
 19     0
 20     0
 21     0
 22     0
 23     0
 24     0
 25     0
 26     0
 27     0
 28     0
 29     0
       ..
 970    0
 971    0
 972    0
 973   